# DF + LIB

In [1]:
# df that have:
# user as numeric feature (from 0 to 5) 
%store -r merge_unique_numain_df

In [2]:
# package that we need
import keras
from keras.models import Sequential
from keras.layers import Dense, Activation
from keras.optimizers import Adam
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.metrics import MeanAbsolutePercentageError
from tensorflow.keras.regularizers import l2

#for the CROSS-VALIDATION
from sklearn.model_selection import KFold

# essential
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

# not necessary in this case
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import mean_squared_error
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay,accuracy_score, precision_score, recall_score,f1_score

# CROSS VALIDATION 

we want to try this approach in order to optimize our recomandation system, more in detail our NN

# GENARALIZE CRSS VALIDATION - all users

In [3]:
# TARGET_var
target='valence'

#FEATURES
features=[    'spotify_track_duration_minute', 'danceability', 'energy', 'key', 'loudness',
    'mode', 'speechiness', 'acousticness', 'tempo', 'time_signature',
    'spotify_track_popularity', 'instrumentalness', 'liveness']

## DEFINE FUNCTIONS

In [4]:
# function to create the model
def create_model_user(shape_x_train): #number of columns -> shape
    
    
    '''model_user_i = Sequential([
        Dense(32, input_dim=shape_x_train, activation='relu', name='first_layer'),
        # 64 as the number of initial layer is a common choice when we build a NN --> VEDIAMO COME FUNZIA POI NEL CASO SI SISTEMA IN SEGUITO
        # input_dim: take the dimension of the x_train array --> that is equivalent to the number of feature in input
        
        # SI POSSONO AGGIUNGERE PER OGNI LAYER --> Dropout(0.3) per evitare overfitting
        

        Dense(16, activation='relu', name='secod_layer'),
        Dense(1, activation='linear', name='final_layer')
    ])'''

    model_user_i = Sequential([
        
        Dense(32,input_dim=shape_x_train,activation='relu',name='first_layer',kernel_regularizer=l2(0.01)),
        # 64 as the number of initial layer is a common choice when we build a NN --> VEDIAMO COME FUNZIA POI NEL CASO SI SISTEMA IN SEGUITO
        # input_dim: take the dimension of the x_train array --> that is equivalent to the number of feature in input
        
        # SI POSSONO AGGIUNGERE PER OGNI LAYER --> Dropout(0.3) per evitare overfitting
        Dropout(0.3),
        Dense(16,activation='relu',name='secod_layer',kernel_regularizer=l2(0.01)),
        Dropout(0.2),
        #Dense(1,activation='linear',name='final_layer')
        Dense(1,activation='sigmoid',name='final_layer') 
        #ATENZION PROVIAMO SIGMOID PRCHè CI SERVE UN OUTPUT NEL RANGE DI O E 1
    
    ])

    #compile the model
    model_user_i.compile(optimizer = 'adam', loss = 'mse', metrics = ['mae'])

    #summary
    #display(model_user_i.summary())

    #return  the model
    return model_user_i

#function to train the model
def fit_model(model, x_train, y_train, x_test, y_test):

    history_model = model.fit(
        x_train,y_train,
        #validation_split=0.2,
        validation_data = [x_test, y_test],
        epochs = 100,
        #epochs=100,
        batch_size=32,
        #verbose=1
    )

    return history_model

def plot_for_model(history, y_hat, x_test):

    #plot loss
    plt.figure(figsize=(10,6))
    plt.plot(history.history['loss'],color='blue',label='loss')
    plt.plot(history.history['val_loss'],color='red',label='val_loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.title('Loss function through epoch')
    plt.legend()
    plt.grid()
    plt.show()

    #plot mean
    plt.figure(figsize=(10,6))
    plt.plot(history.history['mae'],color='blue',label='mae')
    plt.plot(history.history['val_mae'],color='red',label='val_mae')
    plt.xlabel('Epoch')
    plt.ylabel('mae')
    plt.title('Mae function through epoch')
    plt.grid()
    plt.legend()
    plt.show()   

def return_train_test_for_cross(train_index, test_index, df):
    #extract the feature and target
    x = df[features]
    y = df[target]

    #normalization
    scaler = StandardScaler()
    x[features] = scaler.fit_transform(x[features])
    #display(x)

    x_train, x_test = x.iloc[train_index], x.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]

    #display(x_train)

    return x_train,x_test,y_train,y_test

def reco_function_conf_matrix(y_test, y_hat, threshold):
    
    #initialize dataframe    
    data = {'Original Test': y_test,
        'Prediction Test': y_hat.flatten(),
        }
    df_prediction = pd.DataFrame(data = data)
    df_prediction['Original over Thrashold'] = df_prediction['Original Test'] > threshold
    df_prediction['Predicted over Thrashold'] = df_prediction['Prediction Test'] > threshold
    #display(df_prediction)

    #plot confusion matrix of reccomandation
    cm = confusion_matrix(df_prediction['Original over Thrashold'], df_prediction['Predicted over Thrashold'])

    # METRIX FOR EVALUATE RECCOMANDATION 
    # Extract TP, TN, FP, FN
    TN, FP, FN, TP = cm.ravel()

    # Calculate metrics manually
    accuracy = (TP + TN) / (TP + TN + FP + FN)
    precision = TP / (TP + FP) if (TP + FP) > 0 else 0
    recall = TP / (TP + FN) if (TP + FN) > 0 else 0
    f1 = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0
    
    '''plt.figure()
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=['True', 'False'], yticklabels=['True', 'False'])
    plt.title('Confiusion matrix')
    plt.xlabel('predicted')
    plt.ylabel('actual')
    plt.show()'''

    # Store metrics in a DataFrame
    df_metrics_cm = pd.DataFrame(
        data =
        #'Metric': ['Accuracy', 'Precision', 'Recall', 'F1-score'],
        {'Value': [accuracy, precision, recall, f1]},
        index = ['Accuracy', 'Precision', 'Recall', 'F1-score']
    )

    #display(df_metrics_cm.round(2))

    return df_prediction,df_metrics_cm

## MAIN

In [5]:
dizio_model_user = {} #in this dizio we store the model for each user
dizio_accuracy = {}

# DICTIONARY TO STORE SONG RECOMMENDATION PER USER
reccomandations_songs_per_user={}

#threshold
threshold = 0.7

#value of folder
k = 5

#loop creation user and model
for i in range(0, len(merge_unique_numain_df['User'].unique())): #i indica lo user   
    
    print('---------------------------------------------------------')
    print(f'USER {i}')
    
    #def dizio
    df_metrics_kfold = pd.DataFrame()

    #select user
    mask_user = merge_unique_numain_df['User'] == i
    #display(mask_user)
    user_i_df = merge_unique_numain_df[mask_user]
    #display(user_i_df) 

    kf = KFold(n_splits = k, shuffle= True, random_state= 42)
    score_fold = []

    loop = 0
    for train_index, test_index in kf.split(user_i_df):

        print('---------------------------------------------------------')
        print(f'K {loop} FOLD') #print(f'K {} FOLD = test set')
        loop = loop + 1

        #return train and test
        x_train,x_test,y_train,y_test = return_train_test_for_cross(train_index, test_index, user_i_df) #da modificare

        #creation model for the specific user
        model_user_i = create_model_user(x_train.shape[1])
        
        #fit the model with the different x, y train (for different user)
        history_model_user_i = fit_model(model_user_i, x_train, y_train, x_test, y_test)

        y_hat = model_user_i.predict(x_test)

        test_loss, test_mae = model_user_i.evaluate(x_test,y_test,verbose=1)
        score_fold.append(test_mae)

        #reccomandation
        reccomandation_df = reco_function_conf_matrix(y_test, y_hat, threshold)
        df_metrics_kfold = pd.concat([df_metrics_kfold,reccomandation_df[1]], axis = 1)
        
    dizio_model_user[f"User{i}"] = {#'model_user_i' : model_user_i,
                                    #'history_model_user_i': history_model_user_i,
                                    #'x_train': x_train,
                                    #'x_test': x_test,
                                    #'y_train': y_train,
                                    #'y_test': y_test,
                                    #'y_hat': y_hat,
                                    #'recommendations:': reccomandation_df
                                    'score_fold': score_fold,
                                    'df_metrics_kfold_mean': pd.DataFrame(data = df_metrics_kfold.mean(axis=1), columns=['Mean_Metrics'])
                                    } 
    #save accuracy 
    dizio_accuracy[f'User{i}'] = {
        #'MAE' : mean_absolute_error(y_test, y_hat),
        #'MSE' : mean_squared_error(y_test, y_hat)
        'Mean MAE': np.mean(score_fold)
    }

---------------------------------------------------------
USER 0
---------------------------------------------------------
K 0 FOLD
Epoch 1/100


C:\Users\lovoa\AppData\Local\Temp\ipykernel_2608\3718103427.py:88: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  x[features] = scaler.fit_transform(x[features])
c:\Users\lovoa\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


143/143 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - loss: 0.3674 - mae: 0.2045 - val_loss: 0.1728 - val_mae: 0.1700
Epoch 2/100
143/143 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.1451 - mae: 0.1742 - val_loss: 0.0777 - val_mae: 0.1593
Epoch 3/100
143/143 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0713 - mae: 0.1655 - val_loss: 0.0491 - val_mae: 0.1542
Epoch 4/100
143/143 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0498 - mae: 0.1607 - val_loss: 0.0412 - val_mae: 0.1500
Epoch 5/100
143/143 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.0445 - mae: 0.1577 - val_loss: 0.0392 - val_mae: 0.1498
Epoch 6/100
143/143 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.0425 - mae: 0.1566 - val_loss: 0.0382 - val_mae: 0.1471
Epoch 7/100
143/143 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.0407 - mae: 0.1525 - val_loss: 0.0377 - val_mae: 0.1477
Epoch 8/100
143/143 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.0411 - mae: 0.1543 - val_loss: 0.0378 - val_mae: 0.1482
Epoch 9/100
143/143 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss:

C:\Users\lovoa\AppData\Local\Temp\ipykernel_2608\3718103427.py:88: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  x[features] = scaler.fit_transform(x[features])
c:\Users\lovoa\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


143/143 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 0.3802 - mae: 0.1923 - val_loss: 0.1780 - val_mae: 0.1765
Epoch 2/100
143/143 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.1470 - mae: 0.1768 - val_loss: 0.0802 - val_mae: 0.1698
Epoch 3/100
143/143 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0705 - mae: 0.1683 - val_loss: 0.0515 - val_mae: 0.1642
Epoch 4/100
143/143 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.0490 - mae: 0.1611 - val_loss: 0.0437 - val_mae: 0.1569
Epoch 5/100
143/143 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0428 - mae: 0.1559 - val_loss: 0.0416 - val_mae: 0.1545
Epoch 6/100
143/143 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.0415 - mae: 0.1545 - val_loss: 0.0409 - val_mae: 0.1539
Epoch 7/100
143/143 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.0408 - mae: 0.1536 - val_loss: 0.0400 - val_mae: 0.1512
Epoch 8/100
143/143 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.0412 - mae: 0.1549 - val_loss: 0.0401 - val_mae: 0.1520
Epoch 9/100
143/143 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss:

...

63/63 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - loss: 0.4003 - mae: 0.2101 - val_loss: 0.2822 - val_mae: 0.1888
Epoch 2/100
63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.2561 - mae: 0.1889 - val_loss: 0.1852 - val_mae: 0.1775
Epoch 3/100
63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.1696 - mae: 0.1791 - val_loss: 0.1245 - val_mae: 0.1690
Epoch 4/100
63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.1141 - mae: 0.1671 - val_loss: 0.0882 - val_mae: 0.1638
Epoch 5/100
63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0842 - mae: 0.1686 - val_loss: 0.0669 - val_mae: 0.1586
Epoch 6/100
63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0648 - mae: 0.1626 - val_loss: 0.0544 - val_mae: 0.1556
Epoch 7/100
63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0550 - mae: 0.1610 - val_loss: 0.0476 - val_mae: 0.1516
Epoch 8/100
63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0501 - mae: 0.1586 - val_loss: 0.0439 - val_mae: 0.1504
Epoch 9/100
63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0457 - mae: 0.1

C:\Users\lovoa\AppData\Local\Temp\ipykernel_2608\3718103427.py:88: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  x[features] = scaler.fit_transform(x[features])
c:\Users\lovoa\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


63/63 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - loss: 0.4280 - mae: 0.2256 - val_loss: 0.3035 - val_mae: 0.1995
Epoch 2/100
63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.2761 - mae: 0.1964 - val_loss: 0.2010 - val_mae: 0.1897
Epoch 3/100
63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.1793 - mae: 0.1799 - val_loss: 0.1347 - val_mae: 0.1801
Epoch 4/100
63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.1222 - mae: 0.1740 - val_loss: 0.0957 - val_mae: 0.1761
Epoch 5/100
63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0905 - mae: 0.1777 - val_loss: 0.0729 - val_mae: 0.1732
Epoch 6/100
63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.0704 - mae: 0.1753 - val_loss: 0.0595 - val_mae: 0.1695
Epoch 7/100
63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0592 - mae: 0.1716 - val_loss: 0.0519 - val_mae: 0.1661
Epoch 8/100
63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0500 - mae: 0.1635 - val_loss: 0.0472 - val_mae: 0.1621
Epoch 9/100
63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0508 - mae: 0.1

C:\Users\lovoa\AppData\Local\Temp\ipykernel_2608\3718103427.py:88: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  x[features] = scaler.fit_transform(x[features])
c:\Users\lovoa\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


63/63 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - loss: 0.4270 - mae: 0.2351 - val_loss: 0.3000 - val_mae: 0.1969
Epoch 2/100
63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.2746 - mae: 0.1978 - val_loss: 0.1981 - val_mae: 0.1852
Epoch 3/100
63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.1834 - mae: 0.1856 - val_loss: 0.1339 - val_mae: 0.1777
Epoch 4/100
63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.1259 - mae: 0.1793 - val_loss: 0.0947 - val_mae: 0.1723
Epoch 5/100
63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0922 - mae: 0.1763 - val_loss: 0.0715 - val_mae: 0.1679
Epoch 6/100
63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0736 - mae: 0.1740 - val_loss: 0.0578 - val_mae: 0.1640
Epoch 7/100
63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0582 - mae: 0.1656 - val_loss: 0.0495 - val_mae: 0.1596
Epoch 8/100
63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0502 - mae: 0.1581 - val_loss: 0.0452 - val_mae: 0.1569
Epoch 9/100
63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0481 - mae: 0.1

# DISPLAY DICTIONARY ACCURAY

In [6]:
display(dizio_accuracy)

{'User0': {'Mean MAE': 0.14161473512649536},
 'User1': {'Mean MAE': 0.136663818359375},
 'User2': {'Mean MAE': 0.09778598099946975},
 'User3': {'Mean MAE': 0.1348993808031082},
 'User4': {'Mean MAE': 0.13888642191886902}}

# DISPLAY DF METRICS CONFUSION MATRIX

In [7]:
dizio_model_user['User4']['df_metrics_kfold_mean']

,Mean_Metrics
Accuracy,0.771408
Precision,0.787000
Recall,0.529404
F1-score,0.631688


In [8]:
df_metrics_total = pd.concat([dizio_model_user['User0']['df_metrics_kfold_mean'],dizio_model_user['User1']['df_metrics_kfold_mean'],dizio_model_user['User2']['df_metrics_kfold_mean'],dizio_model_user['User3']['df_metrics_kfold_mean'],dizio_model_user['User4']['df_metrics_kfold_mean']], axis=1)
#type(df_metrics_total)
#df_metrics_total.columns(colume_name = ['User0','User1','User2','User3','User4'])
df_metrics_total.columns = ['User0','User1','User2','User3','User4']
display(df_metrics_total)

,User0,User1,User2,User3,User4
Accuracy,0.725209,0.796745,0.818519,0.816636,0.771408
Precision,0.705750,0.793542,0.754991,0.771689,0.787000
Recall,0.521434,0.635598,0.679926,0.334407,0.529404
F1-score,0.592129,0.704249,0.713835,0.465565,0.631688


In [9]:
df_metrics_mean = pd.DataFrame(data = df_metrics_total.mean(axis=1), columns=['Mean_Metrics'])
display(df_metrics_mean.round(3))

,Mean_Metrics
Accuracy,0.786
Precision,0.763
Recall,0.540
F1-score,0.621
